In [1]:
import mailcom.inout
import mailcom.parse
import pandas as pd
from IPython.display import display, HTML

c:\Users\Felix\miniconda3\envs\mailcom\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import files from /data/in

io = mailcom.inout.InoutHandler("../mailcom/test/data")
io.list_of_files()

# create pseudonymization object
ps = mailcom.parse.Pseudonymize()
ps.init_spacy("fr")
ps.init_transformers()

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
# function for displaying the result using HTML
def highlight_ne(text, per_list, org_list, loc_list, misc_list):
    # highlight PER
    for loc in loc_list:
        text = text.replace(loc, "<span style=\"background-color:green\">" + loc + "</span>")
    for org in org_list:
        text = text.replace(org, "<span style=\"background-color:blue\">" + org + "</span>")
    for misc in misc_list:
        text = text.replace(misc, "<span style=\"background-color:yellow\">" + misc + "</span>")
    for per in per_list:
        text = text.replace(per, "<span style=\"background-color:red\">" + per + "</span>")
    return text

In [22]:
# loop over mails and pseudonymize them
out_list = []
for file in io.email_list:
    print("Parsing input file {}".format(file))
    text = io.get_text(file)
    # after this function was called, the email metadata can be accessed via io.email_content
    # the dict already has the entries content, date, attachments, attachment type
    email_dict = io.email_content.copy()
    html_text = io.get_html_text(text)
    email_dict["html_text"] = html_text
    if not text:
        continue
    # Test functionality of Pseudonymize class
    output_text = ps.pseudonymize(html_text)

    # display original text and highlight found and replaced NEs
    highlighted_html = highlight_ne(html_text, ps.per_list, ps.org_list, ps.loc_list, ps.misc_list)
    display(HTML(highlighted_html))

    # add pseudonymized text to dict
    email_dict["pseudo_content"] = output_text
    out_list.append(email_dict)


Parsing input file C:\Users\Felix\Documents\GitHub\mailcom\mailcom\test\data\Bonjour Agathe.eml


Parsing input file C:\Users\Felix\Documents\GitHub\mailcom\mailcom\test\data\Re reunião agendada para o dia 24 de abril 2024-04-17T17_39_49+02 00.eml


Parsing input file C:\Users\Felix\Documents\GitHub\mailcom\mailcom\test\data\Re_ purismo.html


Parsing input file C:\Users\Felix\Documents\GitHub\mailcom\mailcom\test\data\Re_ ¡Voy el 24!.html


In [23]:
# write output to pandas df
df = pd.DataFrame(out_list)

In [24]:
# print results
for idx, mail in df.iterrows():
    print("Email", idx)
    print("Original Text:\n", mail["html_text"])
    print("Pseudonymized Text:\n", mail["pseudo_content"])	

Email 0
Original Text:
 J'espère que tu vas bien! Je voulais partager avec toi quelques photos de mon dernier voyage!
[Inline-Bild]

[Inline-Bild]

À bientôt,

Pierre

Pseudonymized Text:
 J'espère que tu vas bien! Je voulais partager avec toi quelques photos de mon dernier voyage!
[Inline-Bild]

[Inline-Bild]

À bientôt,

Claude

Email 1
Original Text:
 
Olá Lino,Espero que este e-mail te encontre bem. Agradeço a confirmação de sua presença na reunião do dia 24 de abril às 15:00 na InnovaTech Solutions.Com relação à confirmação dos demais membros da equipe, irei verificar a disponibilidade de cada um e te retorno o mais breve possível para que possamos organizar todos os detalhes necessários. A relevância dos temas a serem discutidos para cada área certamente contribuirá para uma sessão produtiva.Até breve,[Alejandro Rodriguez]
InnovaTech Solutions S.L



                        Am Mittwoch, 17. April 2024 um 17:30:19 MESZ hat lino.silva1 <lino.silva1@protonmail.com> Folgendes geschri